In [ ]:
%matplotlib inline
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [ ]:
import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler

Example data:
* An experimental drug was tested on individuals from ages 13 to 100
* The trial had 2100 participants. Half were under 65 years old, half were over 65 years old.
* 95% of patientes 65 or older experienced side effects.
* 95% of patients under 65 experienced no side effects.

In [ ]:
train_labels = []
train_samples = []

for i in range(50):
    # The 5% of younger individuals who did experience side effects
    random_younger =randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)
    
    # The 5% of older individuals who did not experience side effects
    random_older =randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)
    
for i in range(1000):
    # The 95% of younger individuals who did not experience side effects
    random_younger =randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)
    
    # The 95% of older individuals who did experience side effects
    random_older =randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

In [ ]:
print(*train_samples[:100])

In [ ]:
test_labels = []
test_samples = []

for i in range(10):
    # The 5% of younger individuals who did experience side effects
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(1)
    
    # The 5% of older individuals who did not experience side effects
    random_older =randint(65,100)
    test_samples.append(random_older)
    test_labels.append(0)
    
for i in range(200):
    # The 95% of younger individuals who did not experience side effects
    random_younger =randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(0)
    
    # The 95% of older individuals who did experience side effects
    random_older =randint(65,100)
    test_samples.append(random_older)
    test_labels.append(1)

In [ ]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)

In [ ]:
test_labels = np.array(test_labels)
test_samples = np.array(test_samples)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform((train_samples).reshape(-1,1))

In [ ]:
scaler1 = MinMaxScaler(feature_range=(0,1))
scaled_test_samples = scaler1.fit_transform((test_samples).reshape(-1,1))

In [ ]:
print(scaled_train_samples[:10])
print(scaled_train_samples[1].shape)

## Starting with Keras

### Train an ANN wih Keras

In [ ]:
model = Sequential([Dense(16, input_shape=(1,),activation='relu'), 
                    Dense(32, activation='relu'),Dense(2, activation='softmax')])

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(lr=.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# model?

In [ ]:
model.fit(scaled_train_samples, train_labels, batch_size=10, epochs=20, shuffle= True, verbose=2)

### Build a validaion set with Keras

#### One way to do validation

#### Another simpler way

In [ ]:
model.fit(scaled_train_samples, train_labels, validation_split=0.1, batch_size=10, 
          epochs=20, shuffle= True, verbose=2)

### Predict

In [ ]:
predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0)

In [ ]:
print(predictions[:10])

In [ ]:
rounded_predictions = model.predict_classes(scaled_test_samples, batch_size=10, verbose=0)

In [ ]:
print(rounded_predictions[:10])

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(test_labels, rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize = False, title ="Confusion matrix", cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")
        
    print(cm)
    
    
    thresh = cm.max() / 2.
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment = "center", color ="white" if cm[i,j]> thresh else "black" )
        
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
cm_plot_labels = ["no side effects", "had side effects"]
plot_confusion_matrix(cm, cm_plot_labels, title="Confusion Matrix")

### How to save and load Keras model

### model.save()

In [ ]:
model.save('medical_trial_model.h5')

The save function saves:
* The architecture of the model allowing to re-create the model.
* The weights of the model.
* The training configuration (loss, optimizer).
* The state of the optimizer, allowing to resume training exacly where you left off.

In [ ]:
from keras.models import load_model
new_model = load_model("medical_trial_model.h5")

In [ ]:
new_model.summary()

In [ ]:
new_model.get_weights()

In [ ]:
new_model.optimizer

### model.to_json()

In [ ]:
# save as JSON
json_string = model.to_json()

# save as YAML
# yaml_string = model.to_yaml()

In [ ]:
json_string

In [ ]:
# model reconstruction from JSON:
from keras.models import model_from_json
model_architecture = model_from_json(json_string)

# model reconstruction from YAML
#from keras.models import model_from_yaml
#model = model_from_yaml(yaml_string)

In [ ]:
model_architecture.summary()

### model.save_weights()

If you only need to save the weights of a model, you can use the following function to save the weights only.

In [ ]:
model.save_weights("my_model_weights.h5")

In [ ]:
model2 = Sequential([Dense(16, input_shape=(1,),activation="relu"), 
                     Dense(32, activation="relu"), 
                     Dense(2, activation="softmax")])

In [ ]:
model2.load_weights("my_model_weights.h5")

In [ ]:
model2.get_weights()